In [37]:
import pandas as pd
import numpy as np
from jsonToDF import extract_json

### Soybean oil

ID No. 946

### Vitamin K1 epoxide-1,4-diol

ID No. 23496

In [2]:
content_df = extract_json('Content.json')

['id',
 'source_id',
 'source_type',
 'food_id',
 'orig_food_id',
 'orig_food_common_name',
 'orig_food_scientific_name',
 'orig_food_part',
 'orig_source_id',
 'orig_source_name',
 'orig_content',
 'orig_min',
 'orig_max',
 'orig_unit',
 'orig_citation',
 'citation',
 'citation_type',
 'creator_id',
 'updater_id',
 'created_at',
 'updated_at',
 'orig_method',
 'orig_unit_expression',
 'standard_content',
 'preparation_type',
 'export']

In [6]:
content_df

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
0,1,1,Nutrient,4,29,Kiwi,Actinidia chinensis PLANCHON [Actinidiaceae],Fruit,FAT,FAT,...,DATABASE,None,None,2014-11-05T13:42:11.000Z,2020-04-27T16:20:52.000Z,None,None,1955.0,raw,0
1,2,1,Nutrient,6,53,Onion,Allium cepa L. [Liliaceae],Bulb,FAT,FAT,...,DATABASE,None,None,2014-11-05T13:42:11.000Z,2020-04-27T16:20:52.000Z,None,None,1853.95,raw,0
2,3,1,Nutrient,6,53,Onion,Allium cepa L. [Liliaceae],Leaf,FAT,FAT,...,DATABASE,None,None,2014-11-05T13:42:11.000Z,2020-04-27T16:20:52.000Z,None,None,4150.0,raw,0
3,4,1,Nutrient,9,55,Chives,Allium schoenoprasum L. [Liliaceae],Leaf,FAT,FAT,...,DATABASE,None,None,2014-11-05T13:42:11.000Z,2020-04-27T16:20:52.000Z,None,None,3900.0,raw,0
4,5,1,Nutrient,11,70,Cashew,Anacardium occidentale L. [Anacardiaceae],Fruit,FAT,FAT,...,DATABASE,None,None,2014-11-05T13:42:11.000Z,2020-04-27T16:20:52.000Z,None,None,2500.0,other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5691006,6180503,30807,Compound,268,None,Jiaogulan beer,None,None,None,None,...,ARTICLE,None,None,2020-04-29T22:59:30.000Z,2020-04-29T22:59:30.000Z,None,None,1.08,"beverage, alcoholic",1
5691007,6180504,31057,Compound,268,None,Guanshan beer,None,None,None,None,...,ARTICLE,None,None,2020-04-29T22:59:30.000Z,2020-04-29T22:59:30.000Z,None,None,0.27,"beverage, alcoholic",1
5691008,6180505,31057,Compound,268,None,Jiaogulan beer,None,None,None,None,...,ARTICLE,None,None,2020-04-29T22:59:30.000Z,2020-04-29T22:59:30.000Z,None,None,0.54,"beverage, alcoholic",1
5691009,6180506,31026,Compound,268,None,Guanshan beer,None,None,None,None,...,ARTICLE,None,None,2020-04-29T22:59:30.000Z,2020-04-29T22:59:30.000Z,None,None,0.05,"beverage, alcoholic",1


In [29]:
vitaminK1_in_soybean_oil = content_df[(content_df['food_id'] == 946) & (content_df['source_type'] == 'Compound') 
           & (content_df['source_id'] == 23496)]['orig_content'].tolist()
vitaminK1_in_soybean_oil

['90.95', '82.35']

In [31]:
vitaminK1_in_soybean_oil = list(map(lambda x: float(x), vitaminK1_in_soybean_oil))
np.mean(vitaminK1_in_soybean_oil)

86.65

In [38]:
foods = extract_json('Food.json')

In [35]:
foods

,id,name,name_scientific,description,itis_id,wikipedia_id,picture_file_name,picture_content_type,picture_file_size,picture_updated_at,...,food_type,created_at,updated_at,creator_id,updater_id,export_to_afcdb,category,ncbi_taxonomy_id,export_to_foodb,public_id
0,1,Angelica,Angelica keiskei,Angelica is a genus of about 60 species of tal...,,Angelica,1.jpg,image/jpeg,111325,2012-04-20T09:29:57.000Z,...,Type 1,2011-02-09T00:37:14.000Z,2019-05-14T18:04:13.000Z,None,2,False,specific,357850,True,FOOD00001
1,2,Savoy cabbage,Brassica oleracea var. sabauda,Savoy cabbage (Brassica oleracea convar. capit...,None,Savoy cabbage,2.jpg,image/jpeg,155178,2012-04-20T09:39:54.000Z,...,Type 1,2011-02-09T00:37:15.000Z,2019-05-14T18:04:13.000Z,None,None,False,specific,1216010,True,FOOD00002
2,3,Silver linden,Tilia argentea,Tilia tomentosa (Silver Lime in the UK and Sil...,845789,Tilia tomentosa,3.jpg,image/jpeg,56367,2012-04-20T09:41:25.000Z,...,Type 1,2011-02-09T00:37:15.000Z,2019-05-17T16:19:45.000Z,None,None,False,specific,None,True,FOOD00003
3,4,Kiwi,Actinidia chinensis,"The kiwifruit, often shortened to kiwi in many...",506775,Kiwifruit,4.jpg,image/jpeg,110661,2012-04-20T09:32:21.000Z,...,Type 1,2011-02-09T00:37:15.000Z,2019-05-14T18:04:13.000Z,None,None,False,specific,3625,True,FOOD00004
4,5,Allium,Allium,Allium haematochiton is a species of wild onio...,42634,Allium haematochiton,5.jpg,image/jpeg,341911,2012-04-20T09:37:44.000Z,...,Type 1,2011-02-09T00:37:15.000Z,2019-05-14T18:04:13.000Z,None,None,False,specific,4678,True,FOOD00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,1020,Cape gooseberry,Physalis peruviana,"Physalis peruviana, a plant species of the gen...",None,None,Cape_gooseberry.jpg,image/jpeg,10136,2020-04-17T16:23:42.000Z,...,Type 1,2020-01-24T17:05:37.000Z,2020-04-17T16:23:43.000Z,None,None,False,None,None,True,FOOD00988
988,1021,Herbal tea,None,Herbal teas are the beverages made from the in...,None,None,Herbal_tea.jpg,image/jpeg,268769,2020-04-17T16:23:43.000Z,...,Type 1,2020-03-12T20:55:17.000Z,2020-04-17T16:23:43.000Z,None,None,False,None,None,True,FOOD00989
989,1022,Fish oil,None,Fish oil is oil derived from the tissues of oi...,None,None,Fish_oil.jpg,image/jpeg,14684,2020-04-17T16:23:43.000Z,...,Type 1,2020-03-12T21:00:02.000Z,2020-04-17T16:23:44.000Z,None,None,False,None,None,True,FOOD00990
990,1023,Taco shell,None,The hard-shell or crispy taco is a tradition t...,None,None,Taco_shell.jpg,image/jpeg,62320,2020-04-17T16:23:44.000Z,...,Type 2,2020-03-12T21:01:58.000Z,2020-04-17T16:23:44.000Z,None,None,False,None,None,True,FOOD00991
